In [2]:
!pip install polars openpyxl

import boto3
import pandas as pd
import polars as pl
import gzip
import tempfile
import time
import os
from tqdm import tqdm
from IPython.display import display
from io import BytesIO

def read_gz_files(s3_bucket, s3_prefix, user_id_column):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)
    dataframes = {}
    
    start_time = time.time()
    
    with tqdm(total=len(response['Contents']), desc=f"Reading {s3_prefix} files", unit="file") as pbar:
        for obj in response['Contents']:
            if obj['Key'].endswith('.gz'):
                file_obj = s3.get_object(Bucket=s3_bucket, Key=obj['Key'])
                file_content = file_obj['Body'].read()
                
                with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                    temp_file.write(gzip.decompress(file_content))
                    temp_file_path = temp_file.name
                
                pandas_df = pd.read_pickle(temp_file_path)
                df = pl.from_pandas(pandas_df)
                df = df.with_columns(pl.col("total_ig_cycle").cast(pl.Int64))
                
                user_id = df[user_id_column][0]
                if user_id not in dataframes:
                    dataframes[user_id] = []
                dataframes[user_id].append(df)
                
                # 一時ファイルを明示的に削除
                os.remove(temp_file_path)
                
                pbar.update(1)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time for reading {s3_prefix} files: {elapsed_time:.2f} seconds")
    
    return dataframes

def read_excel_from_s3(s3_uri):
    s3 = boto3.client('s3')
    bucket_name, key_name = s3_uri.replace("s3://", "").split("/", 1)
    obj = s3.get_object(Bucket=bucket_name, Key=key_name)
    data = obj['Body'].read()
    
    excel_df = pd.read_excel(BytesIO(data), sheet_name="故障・仮故障", usecols=["VIN", "故障ig_cycle"])
    return excel_df

def extract_failure_data(failure_df, failure_range_list):
    extracted_data = {}
    vin_counts = {}
    
    for _, row in failure_range_list.iterrows():
        vin = row["VIN"]
        failure_ig_cycle = row["故障ig_cycle"]
        
        if vin in failure_df:
            user_df = pl.concat(failure_df[vin])
            user_df = user_df.with_columns(pl.col("total_ig_cycle").cast(pl.Int64))
            
            start_cycle = failure_ig_cycle - 20
            end_cycle = failure_ig_cycle
            
            mask = (user_df["total_ig_cycle"] >= start_cycle) & (user_df["total_ig_cycle"] <= end_cycle)
            extracted_user_df = user_df.filter(mask)
            
            if not extracted_user_df.is_empty():
                if vin not in vin_counts:
                    vin_counts[vin] = 1
                    extracted_data[vin] = extracted_user_df
                else:
                    vin_counts[vin] += 1
                    new_vin = f"{vin}-{vin_counts[vin]}"
                    extracted_data[new_vin] = extracted_user_df
    
    return extracted_data

def print_extracted_ig_cycles(extracted_data):
    for vin, user_df in extracted_data.items():
        ig_cycles = user_df["total_ig_cycle"].unique().to_list()
        print(f"VIN: {vin}, Extracted IG Cycles: {ig_cycles}")




[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# S3のバケット名とプレフィックス
s3_bucket = "hm-prd-s3-vase-vase-wmxc-tmp"
normal_prefix = "wvwc/J0114690/EGT_v4/01_normal"
anomaly_prefix = "wvwc/J0114690/EGT_v4/02_anormal/02_prebroke"

# ユーザーID列の名前
user_id_column = "hashvin"

# 正常データの読み込み
normal_dataframes = read_gz_files(s3_bucket, normal_prefix, user_id_column)

# 異常データの読み込み
anomaly_dataframes = read_gz_files(s3_bucket, anomaly_prefix, user_id_column)

print(f"Number of normal users: {len(normal_dataframes)}")
print(f"Number of anomaly users: {len(anomaly_dataframes)}")

Reading wvwc/J0114690/EGT_v4/01_normal files: 100%|██████████| 962/962 [04:17<00:00,  3.74file/s]


Elapsed time for reading wvwc/J0114690/EGT_v4/01_normal files: 257.53 seconds


Reading wvwc/J0114690/EGT_v4/02_anormal/02_prebroke files: 100%|██████████| 21/21 [00:06<00:00,  3.17file/s]

Elapsed time for reading wvwc/J0114690/EGT_v4/02_anormal/02_prebroke files: 6.63 seconds
Number of normal users: 962
Number of anomaly users: 21


In [4]:
# 故障ig範囲リストの読み込み
excel_s3_uri = "s3://hm-prd-s3-vase-vase-wmxc-tmp/wvwc/J0114690/EGT_v4/故障タイプ一覧_d5.xlsx"
failure_range_list = read_excel_from_s3(excel_s3_uri)

# 故障データの抽出
extracted_failure_data = extract_failure_data(anomaly_dataframes, failure_range_list)

# 抽出されたIGサイクルの出力
print_extracted_ig_cycles(extracted_failure_data)

VIN: A057, Extracted IG Cycles: [314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334]
VIN: A075, Extracted IG Cycles: [487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 502, 504, 506, 507]
VIN: A075-2, Extracted IG Cycles: [416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 429, 430, 431, 432, 433, 435, 436]
VIN: A077, Extracted IG Cycles: [142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162]
VIN: A088, Extracted IG Cycles: [59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79]
VIN: A106, Extracted IG Cycles: [66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78]
VIN: A191, Extracted IG Cycles: [109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129]
VIN: A194, Extracted IG Cycles: [319, 320, 321, 322, 323, 324, 326, 327, 328, 329, 330, 331, 332, 333, 334]
VIN: A212, Extracted IG Cycles: [397, 398

In [19]:
import polars as pl

def count_data_states(dataframes):
    data_states = {
        "null": [],
        "nan": [],
        "int": 0,
        "float": [],
    }
    
    total_dc_count = 0
    
    for vin, user_dfs in dataframes.items():
        for df in user_dfs:
            total_dc_count += len(df)
            
            null_mask = df["total_ig_cycle"].is_null()
            if null_mask.any():
                data_states["null"].extend([(vin, cycle) for cycle in df[null_mask]["total_ig_cycle"]])
            
            nan_mask = df["total_ig_cycle"].is_nan()
            if nan_mask.any():
                data_states["nan"].extend([(vin, cycle) for cycle in df[nan_mask]["total_ig_cycle"]])
            
            if df["total_ig_cycle"].dtype == pl.Int64:
                data_states["int"] += len(df)
            
            float_mask = df["total_ig_cycle"].is_float()
            if float_mask.any():
                data_states["float"].extend([(vin, cycle) for cycle in df[float_mask]["total_ig_cycle"]])
    
    return data_states, total_dc_count

# 正常データのデータ状態をカウント
normal_data_states, normal_total_dc_count = count_data_states(normal_dataframes)

# 故障データのデータ状態をカウント
failure_data_states, failure_total_dc_count = count_data_states(extracted_failure_data)

# 結果を表形式で出力
print("Data State\tNormal Count\tNormal VIN-Cycle\tFailure Count\tFailure VIN-Cycle")
for state in ["null", "nan", "int", "float"]:
    normal_count = normal_data_states[state] if state == "int" else len(normal_data_states[state])
    failure_count = failure_data_states[state] if state == "int" else len(failure_data_states[state])
    
    normal_vin_cycle = ", ".join([f"({vin}, {cycle})" for vin, cycle in normal_data_states[state]]) if state != "int" else "-"
    failure_vin_cycle = ", ".join([f"({vin}, {cycle})" for vin, cycle in failure_data_states[state]]) if state != "int" else "-"
    
    print(f"{state}\t{normal_count}\t{normal_vin_cycle}\t{failure_count}\t{failure_vin_cycle}")

print(f"\nTotal DC Count (Normal): {normal_total_dc_count}")
print(f"Total DC Count (Failure): {failure_total_dc_count}")

/tmp/ipykernel_341/3801413405.py:28: DeprecationWarning: `is_float` is deprecated. Use `Series.dtype.is_float()` instead.
  float_mask = df["total_ig_cycle"].is_float()


AttributeError: 'bool' object has no attribute 'any'

In [5]:
import polars as pl
from scipy.stats import wasserstein_distance
from tqdm import tqdm
import psutil
import matplotlib.pyplot as plt
from datetime import datetime
import multiprocessing
import os

pl.enable_string_cache()

# ヒストグラムのビン数
num_bins = 10

# 高類似度正常リストのサイズ
X = sum(len(df) for df in extracted_failure_data.values()) * 10

# 車速データのラベル
speed_label = 'vehiclespeed_eat'

# メモリ使用状況を表示する関数
def display_current_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    current_memory_usage = memory_info.rss / (1024 ** 2)  # 現在の使用量をMBで取得
    print(f"Current Memory Usage: {current_memory_usage:.2f} MB")

# 故障データのヒストグラムを計算
failure_hist_data = []
for failure_vin, failure_df in tqdm(extracted_failure_data.items(), desc="Calculating failure histograms"):
    failure_hist_df = failure_df.lazy().groupby('total_ig_cycle').agg(
        pl.col(speed_label).hist(bins=num_bins).alias('failure_hist')
    ).with_columns(
        pl.lit(failure_vin).alias('failure_vin'),
        pl.col('total_ig_cycle').cast(pl.Int64).alias('failure_ig_cycle')  # Cast to Int64
    ).collect()
    failure_hist_data.append(failure_hist_df)

failure_hist_df = pl.concat(failure_hist_data)
display_current_memory_usage()

# 正常データのヒストグラムを計算
normal_hist_data = []
for normal_vin, normal_user_dfs in tqdm(normal_dataframes.items(), desc="Calculating normal histograms"):
    for normal_df in normal_user_dfs:
        normal_hist_df = normal_df.lazy().groupby('total_ig_cycle').agg(
            pl.col(speed_label).hist(bins=num_bins).alias('normal_hist')
        ).with_columns(
            pl.lit(normal_vin).alias('normal_vin'),
            pl.col('total_ig_cycle').cast(pl.Int64).alias('normal_ig_cycle')  # Cast to Int64
        ).collect()
        normal_hist_data.append(normal_hist_df)

normal_hist_df = pl.concat(normal_hist_data)
display_current_memory_usage()

# 類似度スコアを計算する関数
def calculate_similarity_score(args):
    normal_hist, failure_hist = args
    return wasserstein_distance(normal_hist, failure_hist)

# 類似度スコアを並列計算
with multiprocessing.Pool(processes=os.cpu_count()) as pool:
    similarity_scores = []
    for normal_row in tqdm(normal_hist_df.iter_rows(named=True), total=len(normal_hist_df), desc="Calculating similarity scores"):
        normal_vin = normal_row['normal_vin']
        normal_ig_cycle = normal_row['normal_ig_cycle']
        normal_hist = normal_row['normal_hist']
        
        args_list = [(normal_hist, failure_row['failure_hist']) for failure_row in failure_hist_df.iter_rows(named=True)]
        scores = pool.map(calculate_similarity_score, args_list)
        
        for i, score in enumerate(scores):
            failure_row = failure_hist_df.row(i, named=True)  # Get the row as a named tuple
            similarity_scores.append({
                'normal_vin': normal_vin,
                'normal_ig_cycle': normal_ig_cycle,
                'failure_vin': failure_row['failure_vin'],
                'failure_ig_cycle': failure_row['failure_ig_cycle'],
                'similarity_score': score
            })

display_current_memory_usage()

# 類似度スコアをCSVファイルに保存
similarity_scores_df = pl.DataFrame(similarity_scores)
csv_file_name = 'similarity_scores.csv'
csv_file_path = os.path.join(os.getcwd(), csv_file_name)

# 同名のCSVファイルがある場合は連番をつける
counter = 1
while os.path.exists(csv_file_path):
    csv_file_name = f'similarity_scores_{counter}.csv'
    csv_file_path = os.path.join(os.getcwd(), csv_file_name)
    counter += 1

similarity_scores_df.write_csv(csv_file_path)

print(f"Similarity scores saved to: {csv_file_path}")


Calculating failure histograms:   0%|          | 0/22 [00:00<?, ?it/s]/tmp/ipykernel_17/1435775627.py:31: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  failure_hist_df = failure_df.lazy().groupby('total_ig_cycle').agg(
Calculating failure histograms: 100%|██████████| 22/22 [00:00<00:00, 910.37it/s]


Current Memory Usage: 31831.18 MB


Calculating normal histograms:   0%|          | 0/962 [00:00<?, ?it/s]/tmp/ipykernel_17/1435775627.py:46: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  normal_hist_df = normal_df.lazy().groupby('total_ig_cycle').agg(
Calculating normal histograms: 100%|██████████| 962/962 [00:01<00:00, 543.37it/s]

Current Memory Usage: 31908.65 MB



Calculating similarity scores: 100%|██████████| 118108/118108 [12:27<00:00, 158.02it/s]


Current Memory Usage: 47537.22 MB
Similarity scores saved to: /root/user/j0114690/similarity_scores_1.csv


In [22]:
import os
import polars as pl

X = sum(len(df) for df in extracted_failure_data.values()) * 10
print(f"Count_X=: {X} ")


# 類似度スコアの上位X個を選択
try:
    if similarity_scores:
        top_scores = sorted(similarity_scores, key=lambda x: x['similarity_score'])[:X]
    else:
        csv_file_path = os.path.join(os.getcwd(), 'similarity_scores.csv')
        if os.path.exists(csv_file_path):
            print(f"Importing similarity scores from {csv_file_path}")
            similarity_scores_df = pl.read_csv(csv_file_path)
            similarity_scores = similarity_scores_df.to_dicts()
            top_scores = sorted(similarity_scores, key=lambda x: x['similarity_score'])[:X]
        else:
            print("No similarity scores found. Please calculate the similarity scores first.")
            top_scores = []
except NameError:
    csv_file_path = os.path.join(os.getcwd(), 'similarity_scores.csv')
    if os.path.exists(csv_file_path):
        print(f"Importing similarity scores from {csv_file_path}")
        similarity_scores_df = pl.read_csv(csv_file_path)
        similarity_scores = similarity_scores_df.to_dicts()
        top_scores = sorted(similarity_scores, key=lambda x: x['similarity_score'])[:X]
    else:
        print("No similarity scores found. Please calculate the similarity scores first.")
        top_scores = []


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 高類似度正常リストの各VIN, DCに対応するデータを抽出
high_similarity_normal_data = []
for row in tqdm(top_scores, desc="Extracting high similarity normal data"):
    normal_vin = row['normal_vin']
    normal_ig_cycle = row['normal_ig_cycle']
    failure_vin = row['failure_vin']
    failure_ig_cycle = row['failure_ig_cycle']
    similarity_score = row['similarity_score']
    
    normal_df = pl.concat(normal_dataframes[normal_vin]).filter(pl.col('total_ig_cycle') == normal_ig_cycle)
    normal_df = normal_df.with_columns(
        pl.lit(failure_vin).alias('failure_vin'),
        pl.lit(failure_ig_cycle).alias('failure_ig_cycle'),
        pl.lit(similarity_score).alias('similarity_score')
    )
    high_similarity_normal_data.append(normal_df)

high_similarity_normal_df = pl.concat(high_similarity_normal_data)
display_current_memory_usage()

# 車速の類似度の高い上位36個の正常DC時系列と故障DC時系列をグラフ化
fig, axes = plt.subplots(6, 6, figsize=(20, 20))
axes = axes.flatten()

for i, row in enumerate(top_scores[:36]):
    normal_vin = row['normal_vin']
    normal_ig_cycle = row['normal_ig_cycle']
    failure_vin = row['failure_vin']
    failure_ig_cycle = row['failure_ig_cycle']
    similarity_score = row['similarity_score']
    
    normal_df = high_similarity_normal_df.filter((pl.col('normal_vin') == normal_vin) & (pl.col('normal_ig_cycle') == normal_ig_cycle))
    failure_df = extracted_failure_data[failure_vin].filter(pl.col('total_ig_cycle') == failure_ig_cycle)
    
    normal_df = normal_df.with_columns(
        pl.col('tsu_current_time').apply(lambda x: datetime.fromisoformat(x.split('+')[0])).alias('timestamp')
    )
    failure_df = failure_df.with_columns(
        pl.col('tsu_current_time').apply(lambda x: datetime.fromisoformat(x.split('+')[0])).alias('timestamp')
    )
    
    normal_df = normal_df.with_columns(
        (pl.col('timestamp') - pl.col('timestamp').min()).dt.seconds.alias('elapsed_time')
    )
    failure_df = failure_df.with_columns(
        (pl.col('timestamp') - pl.col('timestamp').min()).dt.seconds.alias('elapsed_time')
    )
    
    ax = axes[i]
    ax.plot(normal_df['elapsed_time'], normal_df[speed_label], label='Normal')
    ax.plot(failure_df['elapsed_time'], failure_df[speed_label], label='Failure')
    ax.set_title(f"Similarity Score: {similarity_score:.2f}")
    ax.set_xlabel('Elapsed Time (seconds)')
    ax.set_ylabel('Vehicle Speed')
    ax.legend()

plt.tight_layout()
plt.show()

Extracting high similarity normal data:  90%|█████████ | 918981/1016260 [08:11<00:51, 1880.51it/s]